In [4]:
import pandas as pd

fire = pd.read_csv("8d76ac6b-5ae8-4428-82a4-043130d17b02.csv")
crime = pd.read_csv("e03a89dd-134a-4ee8-a2bd-62c40aeebc6f.csv")
firearm = pd.read_csv("e967381d-d7e9-48e3-a2a2-39262f7fa5c4.csv")
pop = pd.read_csv("82f29015-6905-4b1c-8300-afe9bb2231b3.csv")

def detect_column(df, target):
    return next((col for col in df.columns if target in col.lower()), None)

fire_counts = fire[detect_column(fire, "neigh")].value_counts().reset_index()
fire_counts.columns = ["Neighborhood", "Fire_Incidents"]

crime_col = detect_column(crime, "neigh")
crime_counts = crime[crime_col].value_counts().reset_index()
crime_counts.columns = ["Neighborhood", "Crime_Incidents"]
crime_counts = pd.concat([
    crime_counts,
    pd.DataFrame({"Neighborhood": ["Mt. Oliver", "Troy Hill"], "Crime_Incidents": [2, 6]})
])

firearm_group = firearm.groupby("neighborhood", dropna=False, as_index=False).sum()
firearm_counts = firearm_group[~firearm_group["neighborhood"].isin(["East Carnegie", "New Homestead", "Ridgemont"])]
firearm_counts = firearm_counts[["neighborhood", "total_count"]].rename(columns={
    "neighborhood": "Neighborhood",
    "total_count": "Firearm_Incidents"
})

pop_data = pop[[detect_column(pop, "neigh"), detect_column(pop, "estimate")]].dropna()
pop_data.columns = ["Neighborhood", "Population"]

merged = pop_data.merge(fire_counts, on="Neighborhood", how="left")
merged = merged.merge(crime_counts, on="Neighborhood", how="left")
merged = merged.merge(firearm_counts, on="Neighborhood", how="left")
merged = merged.fillna(0)

merged["Fire_per_capita"] = merged["Fire_Incidents"] / merged["Population"]
merged["Crime_per_capita"] = merged["Crime_Incidents"] / merged["Population"]
merged["Firearm_per_capita"] = merged["Firearm_Incidents"] / merged["Population"]
merged["Average_per_capita"] = merged[["Fire_per_capita", "Crime_per_capita", "Firearm_per_capita"]].mean(axis=1)

safest = merged.sort_values("Average_per_capita").head(10)
print(" Top 10 Safest Neighborhoods Overall (All Incidents Per Capita):")
print(safest[["Neighborhood", "Average_per_capita"]].to_string(index=False))

 Top 10 Safest Neighborhoods Overall (All Incidents Per Capita):
        Neighborhood  Average_per_capita
Mount Oliver Borough            0.001569
          Mt. Oliver            0.010033
 Squirrel Hill North            0.013115
      Swisshelm Park            0.015613
       Regent Square            0.016134
           Troy Hill            0.019857
       New Homestead            0.020898
     Stanton Heights            0.021144
       North Oakland            0.021246
 Squirrel Hill South            0.023688
